In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
# from fastai.text import *   # Quick accesss to NLP functionality
import html
from pathlib import Path

In [3]:
import torch.nn as nn

In [4]:
from fastai.text.models.transformer import tfmer_lm_config, GeLU, init_transformer, MultiHeadAttention
from fastai.text.models.awd_lstm import RNNDropout
from fastai.text.learner import LanguageLearner

In [5]:
PATH=Path.home()/'data/wikitext-2-raw'

In [6]:
from fastai.basic_data import *
from fastai.torch_core import *
from fastai.layers import *

In [7]:
bptt = 256
data = load_data(PATH, bs=8, bptt=bptt)

In [8]:

class LinearDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, tie_encoder:nn.Module=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        decoded = self.decoder(outputs[-1])
        return decoded, raw_outputs, outputs


In [9]:
class DecoderLayer(nn.Module):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, n_heads:int, d_model:int, d_head:int, d_inner:int, bias:bool=True):
        super().__init__()
        self.mhra = MultiHeadAttention(n_heads, d_model, d_head, bias=bias)
        self.ff   = feed_forward(d_model, d_inner)
    
    def forward(self, x:Tensor, mask:Tensor=None, **kwargs): 
        attn = self.mhra(x, mask=mask, **kwargs)
        res = self.ff(attn)
        return res

In [10]:

# def feed_forward(d_model:int, d_ff:int, ff_p:float=0., act=nn.ReLU, double_drop:bool=True):
def feed_forward(d_model:int, d_inner:int, ff_p:float=0.1):
    layers = [
        nn.Linear(d_model, d_inner), 
        GeLU(),
        nn.Linear(d_inner, d_model), 
        nn.Dropout(ff_p), 
        MergeLayer(),
        nn.LayerNorm(d_model)
    ]
    return SequentialEx(*layers)


In [11]:
class DownsampleLayer(nn.Module):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, d_model:int, n_heads:int, d_head:int, ff_p:float=0.1, downsample=False):
        super().__init__()
        
        self.mhra = MultiHeadAttention(n_heads, d_model, d_head, bias=True)
        
        d_inner = d_model*4
        self.downsample = downsample
        if self.downsample:
            self.ln1 = nn.Conv1d(d_model, d_inner, (2), stride=2)
            d_out = d_model * 2
        else:
            self.ln1 = nn.Linear(d_model, d_inner)
            d_out = d_model
        self.act = GeLU()
        self.ln2 = nn.Linear(d_inner, d_out)
        self.drop = nn.Dropout(ff_p)
        self.norm = nn.LayerNorm(d_out)

    
    def forward(self, x:Tensor):
        x_attn = self.mhra(x)
        x = x_attn
        
        if self.downsample:
            x_p = x.permute(0, 2, 1)
            x0_0 = self.ln1(x_p) # bptt x emb x bptt
            x0 = x0_0.permute(0, 2, 1)
        else:
            x0 = self.ln1(x)
        x1 = self.act(x0)
        x2 = self.ln2(x1)
        
        if self.downsample:
            x3 = x0 + x2
        else:
            x3 = x + x2
            
        return self.norm(self.drop(x3))

In [12]:
m = nn.ConvTranspose1d(128, 128, (2), stride=2)

In [13]:
class UpsampleLayer(nn.Module):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, d_model:int, n_heads:int, d_head:int, ff_p:float=0.1, upsample=False):
        super().__init__()
        
        self.mhra = MultiHeadAttention(n_heads, d_model, d_head, bias=True)
        
        d_inner = d_model*4
        self.upsample = upsample
        if self.upsample:
            self.ln1 = nn.ConvTranspose1d(d_model, d_inner, (2), stride=2)
            d_out = d_model // 2
        else:
            self.ln1 = nn.Linear(d_model, d_inner)
            d_out = d_model
        self.act = GeLU()
        self.ln2 = nn.Linear(d_inner, d_out)
        self.drop = nn.Dropout(ff_p)
        self.norm = nn.LayerNorm(d_out)

    
    def forward(self, x:Tensor, x_skip:Tensor):
        x_attn = self.mhra(x)
        x = x_attn
        
        if self.upsample:
            x_p = x.permute(0, 2, 1)
            x0_0 = self.ln1(x_p) # bptt x emb x bptt
            x0 = x0_0.permute(0, 2, 1)
        else:
            x0 = self.ln1(x)
        x1 = self.act(x0)
        x2 = self.ln2(x1)
        
        if self.upsample:
            x3 = x0 + x2
        else:
            x3 = x + x2
            
        x4 = self.norm(self.drop(x3))
        return x4 + x_skip

## Testing decoder layer

In [14]:
bs,bptt,d_model = 4, 64, 128
# d1 = DecoderLayer(n_heads=4, d_model=d_model, d_head=32, d_inner=512)
d1 = DownsampleLayer(n_heads=4, d_model=d_model, d_head=32, downsample=True)
mask = torch.triu(torch.ones(bptt, bptt), diagonal=1).byte()[None,None]
xb = torch.ones(bs,bptt,d_model)

In [15]:
xb.shape, d1(xb).shape

RuntimeError: The size of tensor a (512) must match the size of tensor b (256) at non-singleton dimension 2

In [ ]:
%debug

Reshape and half

In [16]:
a = torch.arange(40).view(2, 4, 5); a

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39]]])

In [17]:
a.view(2, 2, 10)

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]]])

In [18]:
d = nn.Linear(d_model*2, d_model)

In [19]:
d(x1.view(bs, bptt//2, d_model*2)).shape

torch.Size([4, 32, 128])

Conv1d example -

In [25]:
m = nn.Conv1d(128, 128, kernel_size=(3), stride=2, padding=3//2)
input = torch.randn(4, 128, 64)
output = m(input)
input.shape, output.shape

(torch.Size([4, 128, 64]), torch.Size([4, 128, 32]))

In [27]:
m = nn.Conv1d(128, 128, kernel_size=2, stride=2, padding=0)
input = torch.randn(4, 128, 64)
output = m(input)
input.shape, output.shape

(torch.Size([4, 128, 64]), torch.Size([4, 128, 32]))

## Conv1d Transpose Up

In [52]:
m = nn.ConvTranspose1d(5, 5, (9), stride=1, padding=0)
input = torch.randn(1, 5, 8)
output = m(input)
input.shape, output.shape

(torch.Size([1, 5, 8]), torch.Size([1, 5, 16]))

In [30]:
m = nn.ConvTranspose1d(128, 128, (2), stride=2)
input = torch.randn(4, 128, 32)
output = m(input)

In [32]:
input.shape, output.shape

(torch.Size([4, 128, 32]), torch.Size([4, 128, 64]))

### Components

In [27]:
bs,bptt,d_model = 4, 64, 128
# d1 = DecoderLayer(n_heads=4, d_model=d_model, d_head=32, d_inner=512)
mhra = MultiHeadAttention(n_heads=4, d_model=d_model, d_head=32)
ff   = feed_forward(d_model=d_model, d_inner=512)

mask = torch.triu(torch.ones(bptt, bptt), diagonal=1).byte()[None,None]
xb = torch.ones(bs,bptt,d_model)

In [28]:
mask

tensor([[[[0, 1, 1,  ..., 1, 1, 1],
          [0, 0, 1,  ..., 1, 1, 1],
          [0, 0, 0,  ..., 1, 1, 1],
          ...,
          [0, 0, 0,  ..., 0, 1, 1],
          [0, 0, 0,  ..., 0, 0, 1],
          [0, 0, 0,  ..., 0, 0, 0]]]], dtype=torch.uint8)

In [29]:
x_attn = mhra(xb, mask=mask)
x_ff = ff(x_attn)
x_attn.shape, x_ff.shape

(torch.Size([4, 64, 128]), torch.Size([4, 64, 128]))

## Transformer

In [41]:
class Transformer(nn.Module):
    "Transformer model: https://arxiv.org/abs/1706.03762."
    def __init__(self, vocab_sz:int, ctx_len:int, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 embed_p:float=0.1, learned_pos_enc:bool=True, **kwargs):
        super().__init__()
        self.encoder = nn.Embedding(vocab_sz, d_model)
        self.pos_enc = nn.Embedding(ctx_len, d_model) if learned_pos_enc else PositionalEncoding(d_model)
        self.drop_emb = nn.Dropout(embed_p)
#         self.layers = nn.ModuleList([DecoderLayer(n_heads, d_model, d_head, d_inner) for k in range(n_layers)])
        self.a1 = DecoderLayer(n_heads, d_model, d_head, d_inner)
        self.a2 = DecoderLayer(n_heads, d_model, d_head, d_inner)
        self.a3 = DecoderLayer(n_heads, d_model, d_head, d_inner)
        self.a4 = DecoderLayer(n_heads, d_model, d_head, d_inner)
    
    def reset(self): pass
    
    def forward(self, x):
        bs, x_len = x.size()
        pos = torch.arange(0, x_len, device=x.device, dtype=x.dtype)
        inp = self.drop_emb(self.encoder(x) + self.pos_enc(pos)[None]) #.mul_(self.d_model ** 0.5)
        mask = torch.triu(x.new_ones(x_len, x_len), diagonal=1).byte()[None,None]
        #[None,:,:None] for einsum implementation of attention
#         for layer in self.layers: inp = layer(inp, mask=mask)
        x1 = self.a1(inp, mask=mask)
        x2 = self.a2(x1, mask=mask)
        x3 = self.a3(x2, mask=mask)
        x4 = self.a4(x3, mask=mask)
        
        print(x1.shape, x2.shape, x3.shape, x4.shape)
        return ([x4],[x4]) #For the LinearDecoder

In [42]:
vocab_sz = len(data.vocab.itos)
# config = tfmer_lm_config.copy(); config
config = {
    'ctx_len': bptt,
    'n_layers': 4,
    'n_heads': 4,
    'd_model': 128,
    'd_head': 32,
    'd_inner': 512,
}

In [43]:
encoder = Transformer(vocab_sz, **config)
decoder = LinearDecoder(vocab_sz, config['d_model'], tie_encoder=encoder.encoder, bias=False)
model = nn.Sequential(encoder, decoder)
model.reset = lambda: True
model.apply(init_transformer)

Sequential(
  (0): Transformer(
    (encoder): Embedding(39880, 128)
    (pos_enc): Embedding(64, 128)
    (drop_emb): Dropout(p=0.1)
    (a1): DecoderLayer(
      (mhra): MultiHeadAttention(
        (attention): Linear(in_features=128, out_features=384, bias=True)
        (out): Linear(in_features=128, out_features=128, bias=True)
        (drop_att): Dropout(p=0.0)
        (drop_res): Dropout(p=0.0)
        (ln): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (ff): SequentialEx(
        (layers): ModuleList(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GeLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1)
          (4): MergeLayer()
          (5): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (a2): DecoderLayer(
      (mhra): MultiHeadAttention(
        (attention): Linear(in_features=128, out_features=384, bias=True)
        (out): Linear(i

In [34]:
learn = LanguageLearner(data, model)

In [35]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: Creating MTGP constants failed. at /home/ubuntu/pytorch/aten/src/THC/THCTensorRandom.cu:33